In [33]:
import os
import h5py
import numpy as np
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers
from keras.models import Sequential
from keras.layers import Convolution2D, MaxPooling2D, ZeroPadding2D
from keras.layers import Activation, Dropout, Flatten, Dense

- created a data/ folder
- created train/ and validation/ subfolders inside data/
- created cats/ and dogs/ subfolders inside train/ and validation/
- put the cat pictures index 0-999 in data/train/cats
- put the cat pictures index 1000-1400 in data/validation/cats
- put the dogs pictures index 12500-13499 in data/train/dogs
- put the dog pictures index 13500-13900 in data/validation/dogs
So that we have 1000 training examples for each class, and 400 validation examples for each class.
In summary, this is our directory structure:
```
data/
    train/
        dogs/
            dog001.jpg
            dog002.jpg
            ...
        cats/
            cat001.jpg
            cat002.jpg
            ...
    validation/
        dogs/
            dog001.jpg
            dog002.jpg
            ...
        cats/
            cat001.jpg
            cat002.jpg
            ...
```
'''

In [34]:
# path to the model weights files.
weights_path = 'weights/vgg16_weights.h5'
top_model_weights_path = 'fc_model.h5'
train_data_dir = 'data2/train/attempt1'
validation_data_dir = 'data2/validation/attempt1'
nb_train_samples = 870
nb_validation_samples = 260
nb_epoch = 10

# dimensions of our images.
img_width, img_height = 256,256

In [35]:
def save_bottlebeck_features():
    datagen = ImageDataGenerator(rescale=1./255)

    # build the VGG16 network
    model = Sequential()
    model.add(ZeroPadding2D((1, 1), input_shape=(3, img_width, img_height)))

    model.add(Convolution2D(64, 3, 3, activation='relu', name='conv1_1'))
    model.add(ZeroPadding2D((1, 1)))
    model.add(Convolution2D(64, 3, 3, activation='relu', name='conv1_2'))
    model.add(MaxPooling2D((2, 2), strides=(2, 2)))

    model.add(ZeroPadding2D((1, 1)))
    model.add(Convolution2D(128, 3, 3, activation='relu', name='conv2_1'))
    model.add(ZeroPadding2D((1, 1)))
    model.add(Convolution2D(128, 3, 3, activation='relu', name='conv2_2'))
    model.add(MaxPooling2D((2, 2), strides=(2, 2)))

    model.add(ZeroPadding2D((1, 1)))
    model.add(Convolution2D(256, 3, 3, activation='relu', name='conv3_1'))
    model.add(ZeroPadding2D((1, 1)))
    model.add(Convolution2D(256, 3, 3, activation='relu', name='conv3_2'))
    model.add(ZeroPadding2D((1, 1)))
    model.add(Convolution2D(256, 3, 3, activation='relu', name='conv3_3'))
    model.add(MaxPooling2D((2, 2), strides=(2, 2)))

    model.add(ZeroPadding2D((1, 1)))
    model.add(Convolution2D(512, 3, 3, activation='relu', name='conv4_1'))
    model.add(ZeroPadding2D((1, 1)))
    model.add(Convolution2D(512, 3, 3, activation='relu', name='conv4_2'))
    model.add(ZeroPadding2D((1, 1)))
    model.add(Convolution2D(512, 3, 3, activation='relu', name='conv4_3'))
    model.add(MaxPooling2D((2, 2), strides=(2, 2)))

    model.add(ZeroPadding2D((1, 1)))
    model.add(Convolution2D(512, 3, 3, activation='relu', name='conv5_1'))
    model.add(ZeroPadding2D((1, 1)))
    model.add(Convolution2D(512, 3, 3, activation='relu', name='conv5_2'))
    model.add(ZeroPadding2D((1, 1)))
    model.add(Convolution2D(512, 3, 3, activation='relu', name='conv5_3'))
    model.add(MaxPooling2D((2, 2), strides=(2, 2)))

    # load the weights of the VGG16 networks
    # (trained on ImageNet, won the ILSVRC competition in 2014)
    # note: when there is a complete match between your model definition
    # and your weight savefile, you can simply call model.load_weights(filename)
    assert os.path.exists(weights_path), 'Model weights not found (see "weights_path" variable in script).'
    f = h5py.File(weights_path)
    for k in range(f.attrs['nb_layers']):
        if k >= len(model.layers):
            # we don't look at the last (fully-connected) layers in the savefile
            break
        g = f['layer_{}'.format(k)]
        weights = [g['param_{}'.format(p)] for p in range(g.attrs['nb_params'])]
        model.layers[k].set_weights(weights)
    f.close()
    print('Model loaded.')

    generator = datagen.flow_from_directory(
            train_data_dir,
            target_size=(img_width, img_height),
            batch_size=32,
            class_mode=None,
            shuffle=False)
    bottleneck_features_train = model.predict_generator(generator, nb_train_samples)
    np.save(open('bottleneck_features_train.npy', 'w'), bottleneck_features_train)

    generator = datagen.flow_from_directory(
            validation_data_dir,
            target_size=(img_width, img_height),
            batch_size=32,
            class_mode=None,
            shuffle=False)
    bottleneck_features_validation = model.predict_generator(generator, nb_validation_samples)
    np.save(open('bottleneck_features_validation.npy', 'w'), bottleneck_features_validation)
    print "model saved!"


def train_top_model():
    print "\nlet's hit it\n"
    train_data = np.load(open('bottleneck_features_train.npy'))
    train_labels = np.array([0] * (nb_train_samples / 2) + [1] * (nb_train_samples / 2))

    validation_data = np.load(open('bottleneck_features_validation.npy'))
    validation_labels = np.array([0] * (nb_validation_samples / 2) + [1] * (nb_validation_samples / 2))

    model = Sequential()
    model.add(Flatten(input_shape=train_data.shape[1:]))
    model.add(Dense(256, activation='relu', W_regularizer=l2(0.01)))
    model.add(Dropout(0.5))
    model.add(Dense(3, activation='sigmoid'))

    model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['accuracy'])

    model.fit(train_data, train_labels,
              nb_epoch=nb_epoch, batch_size=32,
              validation_data=(validation_data, validation_labels))
    model.save_weights(top_model_weights_path)

In [36]:
save_bottlebeck_features()


Model loaded.
Found 870 images belonging to 3 classes.
Found 260 images belonging to 3 classes.
model saved!


In [22]:
def preprocess_image_batch(image_paths, img_size=None, crop_size=None, color_mode="rgb", out=None):
    img_list = []
    
    for im_path in image_paths:
        img = imread(im_path, mode='RGB')
        if img_size:
            img = imresize(img,img_size)
            
        img = img.astype('float32')
        # We permute the colors to get them in the BGR order
        if color_mode=="bgr":
            img[:,:,[0,1,2]] = img[:,:,[2,1,0]]
        # We normalize the colors with the empirical means on the training set
        img[:, :, 0] -= 123.68 
        img[:, :, 1] -= 116.779
        img[:, :, 2] -= 103.939
        img = img.transpose((2, 0, 1))

        if crop_size:
            img = img[:,(img_size[0]-crop_size[0])//2:(img_size[0]+crop_size[0])//2
                      ,(img_size[1]-crop_size[1])//2:(img_size[1]+crop_size[1])//2]
            
        img_list.append(img)

    img_batch = np.stack(img_list, axis=0)
    if not out is None:
        out.append(img_batch)
    else:
        return img_batch

In [23]:
# build the VGG16 network
model = Sequential()
model.add(ZeroPadding2D((1, 1), input_shape=(3, img_width, img_height)))

model.add(Convolution2D(64, 3, 3, activation='relu', name='conv1_1'))
model.add(ZeroPadding2D((1, 1)))
model.add(Convolution2D(64, 3, 3, activation='relu', name='conv1_2'))
model.add(MaxPooling2D((2, 2), strides=(2, 2)))

model.add(ZeroPadding2D((1, 1)))
model.add(Convolution2D(128, 3, 3, activation='relu', name='conv2_1'))
model.add(ZeroPadding2D((1, 1)))
model.add(Convolution2D(128, 3, 3, activation='relu', name='conv2_2'))
model.add(MaxPooling2D((2, 2), strides=(2, 2)))

model.add(ZeroPadding2D((1, 1)))
model.add(Convolution2D(256, 3, 3, activation='relu', name='conv3_1'))
model.add(ZeroPadding2D((1, 1)))
model.add(Convolution2D(256, 3, 3, activation='relu', name='conv3_2'))
model.add(ZeroPadding2D((1, 1)))
model.add(Convolution2D(256, 3, 3, activation='relu', name='conv3_3'))
model.add(MaxPooling2D((2, 2), strides=(2, 2)))

model.add(ZeroPadding2D((1, 1)))
model.add(Convolution2D(512, 3, 3, activation='relu', name='conv4_1'))
model.add(ZeroPadding2D((1, 1)))
model.add(Convolution2D(512, 3, 3, activation='relu', name='conv4_2'))
model.add(ZeroPadding2D((1, 1)))
model.add(Convolution2D(512, 3, 3, activation='relu', name='conv4_3'))
model.add(MaxPooling2D((2, 2), strides=(2, 2)))

model.add(ZeroPadding2D((1, 1)))
model.add(Convolution2D(512, 3, 3, activation='relu', name='conv5_1'))
model.add(ZeroPadding2D((1, 1)))
model.add(Convolution2D(512, 3, 3, activation='relu', name='conv5_2'))
model.add(ZeroPadding2D((1, 1)))
model.add(Convolution2D(512, 3, 3, activation='relu', name='conv5_3'))
model.add(MaxPooling2D((2, 2), strides=(2, 2)))

In [24]:
# load the weights of the VGG16 networks
# (trained on ImageNet, won the ILSVRC competition in 2014)
# note: when there is a complete match between your model definition
# and your weight savefile, you can simply call model.load_weights(filename)
assert os.path.exists(weights_path), 'Model weights not found (see "weights_path" variable in script).'
f = h5py.File(weights_path)
for k in range(f.attrs['nb_layers']):
    if k >= len(model.layers):
        # we don't look at the last (fully-connected) layers in the savefile
        break
    g = f['layer_{}'.format(k)]
    weights = [g['param_{}'.format(p)] for p in range(g.attrs['nb_params'])]
    model.layers[k].set_weights(weights)
f.close()
print('Model loaded.')

Model loaded.


In [56]:
print model.output_shape

(None, 3)


In [72]:
import pydot
import graphviz


In [74]:
from graphviz import Digraph
dot = Digraph(comment='The Round Table')
dot

RuntimeError: failed to execute ['dot', '-Tsvg'], make sure the Graphviz executables are on your systems' path

In [79]:
from keras.utils.visualize_util import plot

AttributeError: 'module' object has no attribute 'find_graphviz'

In [78]:
plot(model, to_file='model.png')

NameError: name 'plot' is not defined

In [67]:
# Adding L2 Regularization
from keras.regularizers import l2
lmbda = 0.01

# build a classifier model to put on top of the convolutional model
top_model = Sequential()
# top_model.add(Flatten(input_shape=model.output_shape[1:]))
# top_model.add(Flatten(input_shape=model.output_shape[1:]))
top_model.add(Dense(3, input_dim=16, activation='softmax',W_regularizer=l2(lmbda)))
# top_model.add(Dense(256, activation='relu', W_regularizer=l2(lmbda)))
top_model.add(Dropout(0.5))
print "layers added"

# note that it is necessary to start with a fully-trained
# classifier, including the top classifier,
# in order to successfully do fine-tuning
top_model.load_weights(top_model_weights_path)
print "weights loaded"

# add the model on top of the convolutional base
model.add(top_model)


# set the first 25 layers (up to the last conv block)
# to non-trainable (weights will not be updated)
for layer in model.layers[:25]:
    layer.trainable = False

TypeError: __init__() takes exactly 1 argument (2 given)

In [ ]:
# compile the model with a SGD/momentum optimizer
# and a very slow learning rate.
model.compile(loss='binary_crossentropy',
              optimizer=optimizers.SGD(lr=1e-4, momentum=0.9),
              metrics=['accuracy'])

In [ ]:
# prepare data augmentation configuration
train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
        train_data_dir,
        target_size=(img_height, img_width),
        batch_size=32,
        class_mode='binary')

validation_generator = test_datagen.flow_from_directory(
        validation_data_dir,
        target_size=(img_height, img_width),
        batch_size=32,
        class_mode='binary')

# fine-tune the model
model.fit_generator(
        train_generator,
        samples_per_epoch=nb_train_samples,
        nb_epoch=nb_epoch,
        validation_data=validation_generator,
        nb_val_samples=nb_validation_samples)

Found 770 images belonging to 2 classes.
Found 250 images belonging to 2 classes.
Epoch 1/50
770/770 [==============================] - 488s - loss: 0.6384 - acc: 0.7792 - val_loss: 0.7686 - val_acc: 0.6280
Epoch 2/50
770/770 [==============================] - 520s - loss: 0.3861 - acc: 0.8195 - val_loss: 0.5246 - val_acc: 0.7640
Epoch 3/50
770/770 [==============================] - 504s - loss: 0.3392 - acc: 0.8351 - val_loss: 0.5448 - val_acc: 0.7400
Epoch 4/50
770/770 [==============================] - 491s - loss: 0.3281 - acc: 0.8455 - val_loss: 0.5721 - val_acc: 0.6840
Epoch 5/50
770/770 [==============================] - 490s - loss: 0.3096 - acc: 0.8714 - val_loss: 0.4415 - val_acc: 0.8000
Epoch 6/50
770/770 [==============================] - 492s - loss: 0.2937 - acc: 0.8714 - val_loss: 0.4726 - val_acc: 0.7640
Epoch 7/50
770/770 [==============================] - 489s - loss: 0.2862 - acc: 0.8636 - val_loss: 0.4884 - val_acc: 0.7440
Epoch 8/50
770/770 [==============================] - 488s - loss: 0.2677 - acc: 0.8740 - val_loss: 0.5361 - val_acc: 0.7520
Epoch 9/50
770/770 [==============================] - 487s - loss: 0.2460 - acc: 0.8974 - val_loss: 0.4113 - val_acc: 0.8280
Epoch 10/50
770/770 [==============================] - 488s - loss: 0.2427 - acc: 0.9000 - val_loss: 0.3701 - val_acc: 0.8280
Epoch 11/50
770/770 [==============================] - 487s - loss: 0.2360 - acc: 0.8948 - val_loss: 0.3862 - val_acc: 0.8280
Epoch 12/50
770/770 [==============================] - 487s - loss: 0.2339 - acc: 0.8974 - val_loss: 0.4228 - val_acc: 0.8160
Epoch 13/50
770/770 [==============================] - 487s - loss: 0.2223 - acc: 0.9078 - val_loss: 0.3836 - val_acc: 0.8280
Epoch 14/50
770/770 [==============================] - 486s - loss: 0.2348 - acc: 0.8987 - val_loss: 0.3973 - val_acc: 0.8200
Epoch 15/50
770/770 [==============================] - 488s - loss: 0.1981 - acc: 0.9143 - val_loss: 0.3746 - val_acc: 0.8120
Epoch 16/50
770/770 [==============================] - 486s - loss: 0.1978 - acc: 0.9195 - val_loss: 0.3471 - val_acc: 0.8400
Epoch 17/50
770/770 [==============================] - 487s - loss: 0.1853 - acc: 0.9260 - val_loss: 0.3412 - val_acc: 0.8880
Epoch 18/50
770/770 [==============================] - 488s - loss: 0.1689 - acc: 0.9182 - val_loss: 0.3098 - val_acc: 0.8640
Epoch 19/50
770/770 [==============================] - 488s - loss: 0.1796 - acc: 0.9247 - val_loss: 0.3428 - val_acc: 0.8240
Epoch 20/50
770/770 [==============================] - 486s - loss: 0.1912 - acc: 0.9208 - val_loss: 0.3643 - val_acc: 0.8800
Epoch 21/50
770/770 [==============================] - 487s - loss: 0.1537 - acc: 0.9506 - val_loss: 0.3131 - val_acc: 0.8880
Epoch 22/50
770/770 [==============================] - 487s - loss: 0.1526 - acc: 0.9390 - val_loss: 0.2786 - val_acc: 0.9040
Epoch 23/50
770/770 [==============================] - 487s - loss: 0.1848 - acc: 0.9195 - val_loss: 0.3315 - val_acc: 0.8760
Epoch 24/50
770/770 [==============================] - 486s - loss: 0.1545 - acc: 0.9351 - val_loss: 0.2830 - val_acc: 0.9080
Epoch 25/50
770/770 [==============================] - 486s - loss: 0.1436 - acc: 0.9481 - val_loss: 0.3036 - val_acc: 0.8880
Epoch 26/50
770/770 [==============================] - 486s - loss: 0.1324 - acc: 0.9584 - val_loss: 0.3680 - val_acc: 0.8520
Epoch 27/50
770/770 [==============================] - 486s - loss: 0.1173 - acc: 0.9623 - val_loss: 0.2787 - val_acc: 0.9200
Epoch 28/50
770/770 [==============================] - 486s - loss: 0.1208 - acc: 0.9610 - val_loss: 0.2671 - val_acc: 0.9320
Epoch 29/50
770/770 [==============================] - 485s - loss: 0.1120 - acc: 0.9584 - val_loss: 0.2915 - val_acc: 0.9240
Epoch 30/50
770/770 [==============================] - 485s - loss: 0.1162 - acc: 0.9597 - val_loss: 0.2824 - val_acc: 0.9000
Epoch 31/50
770/770 [==============================] - 484s - loss: 0.1206 - acc: 0.9494 - val_loss: 0.2306 - val_acc: 0.9120
Epoch 32/50
770/770 [==============================] - 486s - loss: 0.1068 - acc: 0.9623 - val_loss: 0.3281 - val_acc: 0.9160
Epoch 33/50
770/770 [==============================] - 485s - loss: 0.0829 - acc: 0.9701 - val_loss: 0.2849 - val_acc: 0.9240
Epoch 34/50
770/770 [==============================] - 486s - loss: 0.0952 - acc: 0.9636 - val_loss: 0.3164 - val_acc: 0.9120
Epoch 35/50
770/770 [==============================] - 484s - loss: 0.0900 - acc: 0.9701 - val_loss: 0.3076 - val_acc: 0.9280
Epoch 36/50
770/770 [==============================] - 486s - loss: 0.0861 - acc: 0.9688 - val_loss: 0.2894 - val_acc: 0.9240
Epoch 37/50
770/770 [==============================] - 485s - loss: 0.0749 - acc: 0.9740 - val_loss: 0.2794 - val_acc: 0.9240
Epoch 38/50
770/770 [==============================] - 486s - loss: 0.0767 - acc: 0.9766 - val_loss: 0.2994 - val_acc: 0.9120
Epoch 39/50
770/770 [==============================] - 485s - loss: 0.0664 - acc: 0.9818 - val_loss: 0.2928 - val_acc: 0.9200
Epoch 40/50
770/770 [==============================] - 489s - loss: 0.0735 - acc: 0.9779 - val_loss: 0.2415 - val_acc: 0.9320
Epoch 41/50
770/770 [==============================] - 485s - loss: 0.0644 - acc: 0.9779 - val_loss: 0.3124 - val_acc: 0.9200
Epoch 42/50
770/770 [==============================] - 497s - loss: 0.0694 - acc: 0.9753 - val_loss: 0.3409 - val_acc: 0.8960
Epoch 43/50
770/770 [==============================] - 499s - loss: 0.0555 - acc: 0.9870 - val_loss: 0.3099 - val_acc: 0.9200
Epoch 44/50
770/770 [==============================] - 492s - loss: 0.0614 - acc: 0.9818 - val_loss: 0.2941 - val_acc: 0.9320
Epoch 45/50
770/770 [==============================] - 492s - loss: 0.0635 - acc: 0.9805 - val_loss: 0.3076 - val_acc: 0.9200
Epoch 46/50
770/770 [==============================] - 492s - loss: 0.0687 - acc: 0.9766 - val_loss: 0.2123 - val_acc: 0.9480
Epoch 47/50
770/770 [==============================] - 493s - loss: 0.1262 - acc: 0.9519 - val_loss: 0.2916 - val_acc: 0.9120
Epoch 48/50
770/770 [==============================] - 491s - loss: 0.0551 - acc: 0.9870 - val_loss: 0.2838 - val_acc: 0.9120
Epoch 49/50
770/770 [==============================] - 493s - loss: 0.0528 - acc: 0.9857 - val_loss: 0.3344 - val_acc: 0.9160
Epoch 50/50
770/770 [==============================] - 491s - loss: 0.0573 - acc: 0.9831 - val_loss: 0.4105 - val_acc: 0.9120

In [22]:
from scipy.misc import imread, imresize, imsave
im = preprocess_image_batch(['data2/nike-1.jpg'],img_size=(256,256), crop_size=(256,256), color_mode="bgr")

In [9]:
from scipy.misc import imread, imresize, imsave
im = preprocess_image_batch(['data2/nike-1.jpg','data2/nike-2.jpg','data2/nike-3.jpg','data2/nike-4.jpg','data2/nike-5.jpg','data2/nike-6.jpg'],img_size=(256,256), crop_size=(256,256), color_mode="bgr")

In [10]:
# predict on Nike-1.jpg = 1
# predict on Nike-5.jpg = 0.4547
out = model.predict(im)


In [16]:
print out[0][3]

[[ 0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.]]


In [31]:
 print 'data2/nike-{0}.jpg'.format(i)

data2/nike-1.jpg
